## 📌Partie 1 : Open-WebUI - Introduction aux Filtres  

## 🔹 Présentation  

Open-WebUI est une interface qui permet d'interagir avec des modèles d'IA tout en offrant des possibilités de personnalisation grâce aux filtres. Ces filtres permettent d’intercepter, modifier ou analyser les requêtes envoyées au modèle et les réponses générées.  

Dans ce notebook, nous allons explorer la structure d’un filtre Open-WebUI à travers un exemple minimaliste.  

---

## 🏗️ Structure d'un Filtre Open-WebUI  

Un filtre est défini sous forme d’une classe Python et suit une structure spécifique.  

### 📜 Métadonnées du Filtre  

Le fichier commence par un en-tête contenant des métadonnées utiles pour identifier le filtre dans Open-WebUI :  

```python
"""
title: Example Filter
author: open-webui
author_url: https://github.com/open-webui
funding_url: https://github.com/open-webui
version: 0.1
"""
```

Cela permet notamment de préciser le créateur du filtre et sa version.

---

### 🏷️ Définition de la Classe `Filter`  

Le filtre est encapsulé dans une classe `Filter`, qui peut contenir différentes méthodes pour interagir avec les requêtes et les réponses.

```python
class Filter:
```

À l'intérieur, une sous-classe `Valves` est définie en tant que modèle `pydantic`. Cette classe pourrait être utilisée pour gérer des paramètres de configuration spécifiques :  

```python
class Valves(BaseModel):
    pass
```

Puis, dans le constructeur `__init__`, on initialise cette configuration :  

```python
def __init__(self):
    self.valves = self.Valves()
```

---

### 🛠️ Fonction `inlet` : Pré-traitement des Requêtes  

La méthode `inlet` est exécutée avant l’envoi d’une requête au modèle. Elle permet de modifier ou de valider la requête avant son traitement.

```python
def inlet(self, body: dict, __user__: Optional[dict] = None) -> dict:
```

Dans cet exemple, la fonction se contente d'afficher le contenu de la requête avant de la renvoyer inchangée :

```python
print("Start --------------------------------------------------------------------------------------------------------------------")
print(body)
print("End --------------------------------------------------------------------------------------------------------------------")
```

---

### 🔄 Fonction `stream` : Modification en Temps Réel  

La méthode `stream` est utilisée pour intercepter et modifier les réponses en streaming.

```python
def stream(self, event: dict) -> dict:
    return event
```

Ici, la fonction est présente mais ne modifie pas les données.

---

### 📤 Fonction `outlet` : Post-traitement des Réponses  

Enfin, la méthode `outlet` est exécutée après que le modèle a généré une réponse. Elle permet de modifier ou d’analyser la réponse avant qu’elle ne soit affichée.

```python
def outlet(self, body: dict, __user__: Optional[dict] = None) -> dict:
    return body
```

Dans ce cas, la réponse est renvoyée telle quelle.

---

## 📌 Conclusion  

Ce filtre est un exemple minimaliste, mais il montre comment Open-WebUI permet d’intercepter les requêtes et les réponses.  

Dans les prochaines sections, nous verrons comment ajouter des fonctionnalités plus avancées, comme l'enrichissement des requêtes, l'analyse des réponses ou encore l'intégration avec des API externes. 🚀

In [ ]:
"""
title: Example Filter
author: open-webui
author_url: https://github.com/open-webui
funding_url: https://github.com/open-webui
version: 0.1
"""

from pydantic import BaseModel
from typing import Optional


class Filter:
    class Valves(BaseModel):
        pass

    def __init__(self):
        # Indicates custom file handling logic. This flag helps disengage default routines in favor of custom
        # implementations, informing the WebUI to defer file-related operations to designated methods within this class.
        # Alternatively, you can remove the files directly from the body in from the inlet hook
        # self.file_handler = True

        # Initialize 'valves' with specific configurations. Using 'Valves' instance helps encapsulate settings,
        # which ensures settings are managed cohesively and not confused with operational flags like 'file_handler'.
        self.valves = self.Valves()
        pass

    def inlet(self, body: dict, __user__: Optional[dict] = None) -> dict:
        # Modify the request body or validate it before processing by the chat completion API.
        # This function is the pre-processor for the API where various checks on the input can be performed.
        # It can also modify the request before sending it to the API.

        print(
            "Start --------------------------------------------------------------------------------------------------------------------",
        )
        print(body)
        print(
            "End --------------------------------------------------------------------------------------------------------------------"
        )

        return body

    def stream(self, event: dict) -> dict:
        # This is where you modify streamed chunks of model output.
        return event

    def outlet(self, body: dict, __user__: Optional[dict] = None) -> dict:
        # Modify or analyze the response body after processing by the API.
        # This function is the post-processor for the API, which can be used to modify the response
        # or perform additional checks and analytics.
        return body


In [2]:
import requests
from bs4 import BeautifulSoup

        
response = requests.get("https://www.lachainemeteo.com/meteo-france/ville-325/previsions-meteo-saint-denis-aujourdhui")
soup = BeautifulSoup(response.text, "html.parser")
temp = soup.find(class_="quarter-temperature").find(class_="tempe").text

temp

'10°'

In [ ]:
# Modification du corps de la demande

body["messages"] = [
                        {
                                    "role": "system",
                                    "content": f"Tu es l'assistant de IRAE. La température à Paris est de {temp} degrés",
                        },  
                    ] + body["messages"]

## 🚀 Partie 2 : Intégration de ChromaDB pour le RAG  

Dans cette section, nous intégrons **ChromaDB**, une base de données vectorielle qui permettra de récupérer du contenu pertinent en fonction des requêtes des utilisateurs.  

### 📦 Installation et Importation de ChromaDB  

D'abord, nous devons installer ChromaDB :  

```python
!pip install chromadb
```

Ensuite, nous importons la bibliothèque :  

```python
import chromadb
```

---

### 📄 Création d'une Base de Connaissances  

Nous stockons un ensemble de documents dans la base ChromaDB. Ces documents représentent des connaissances spécialisées sur différents sujets.  

```python
docs = [
    """Vous êtes un expert en gastronomie, passionné par la richesse culinaire du monde entier. Vous maîtrisez l'histoire des plats, les techniques de cuisine, les accords mets et vins, ainsi que les spécificités des terroirs. Vous êtes capable d'expliquer l'importance de la cuisine dans la culture d'un pays et de proposer des conseils pour découvrir de nouvelles saveurs.""",

    """Vous êtes un scientifique curieux et engagé dans la découverte du monde. Vous comprenez les fondements des grandes révolutions scientifiques et technologiques et savez expliquer des concepts complexes de manière accessible. Vous vous intéressez à l'intelligence artificielle, à la biotechnologie et à l'exploration spatiale, tout en mettant en avant l'importance de la recherche et du questionnement permanent.""",

    """Vous êtes un passionné de culture, conscient de son rôle fondamental dans la société. Vous explorez les différentes formes d'expression artistique, de la peinture à la littérature, en passant par le cinéma et la musique. Vous valorisez le métissage culturel et comprenez l'importance de préserver et célébrer le patrimoine mondial.""",

    """Vous êtes un analyste du sport et de ses enjeux. Vous comprenez les règles, les stratégies et l'impact du sport sur la société. Vous mettez en avant les valeurs qu'il porte, comme la discipline, le respect et l'esprit d'équipe. Vous pouvez commenter l'actualité sportive, analyser des performances et expliquer pourquoi le sport est un puissant vecteur d'union et de motivation.""",

    """Vous êtes un expert en échecs, passionné par la stratégie et la logique. Vous maîtrisez les ouvertures, les tactiques et les finales. Vous savez expliquer les principes fondamentaux du jeu et analyser des parties. Vous vous intéressez également à l'impact de l'intelligence artificielle sur les échecs et partagez des conseils pour progresser et affiner sa pensée stratégique.""",
]
```

Nous utilisons un **client ChromaDB** en mode persistant pour stocker ces documents :  

```python
client = chromadb.PersistentClient(path="./chromadb")
```

Nous créons ensuite une **collection** pour stocker ces documents :  

```python
collection = client.create_collection("all-my-documents")
```

Nous ajoutons les documents à la collection avec des **métadonnées** et des **IDs uniques** :  

```python
collection.add(
    documents=docs,
    metadatas=[{"source": "notion"}, {"source": "google-docs"}, {"source": "google-docs"}, {"source": "google-docs"}, {"source": "google-docs"}],
    ids=["doc1", "doc2", "doc3", "doc4", "doc5"],
)
```

---

### 🔍 Recherche d'Informations  

Nous pouvons maintenant interroger la base pour récupérer les **documents les plus pertinents** en fonction d'une requête :  

```python
results = collection.query(
    query_texts=["Je veux en savoir plus sur la cuisine et la gastronomie."],
    n_results=2
)
```

Le système retourne les **deux documents les plus pertinents** liés à la gastronomie.

---

In [1]:
import chromadb

# Liste des documents
docs = [
    """Vous êtes un expert en gastronomie, passionné par la richesse culinaire du monde entier. Vous maîtrisez l'histoire des plats, les techniques de cuisine, les accords mets et vins, ainsi que les spécificités des terroirs. Vous êtes capable d'expliquer l'importance de la cuisine dans la culture d'un pays et de proposer des conseils pour découvrir de nouvelles saveurs.""",

    """Vous êtes un scientifique curieux et engagé dans la découverte du monde. Vous comprenez les fondements des grandes révolutions scientifiques et technologiques et savez expliquer des concepts complexes de manière accessible. Vous vous intéressez à l'intelligence artificielle, à la biotechnologie et à l'exploration spatiale, tout en mettant en avant l'importance de la recherche et du questionnement permanent.""",

    """Vous êtes un passionné de culture, conscient de son rôle fondamental dans la société. Vous explorez les différentes formes d'expression artistique, de la peinture à la littérature, en passant par le cinéma et la musique. Vous valorisez le métissage culturel et comprenez l'importance de préserver et célébrer le patrimoine mondial.""",

    """Vous êtes un analyste du sport et de ses enjeux. Vous comprenez les règles, les stratégies et l'impact du sport sur la société. Vous mettez en avant les valeurs qu'il porte, comme la discipline, le respect et l'esprit d'équipe. Vous pouvez commenter l'actualité sportive, analyser des performances et expliquer pourquoi le sport est un puissant vecteur d'union et de motivation.""",

    """Vous êtes un expert en échecs, passionné par la stratégie et la logique. Vous maîtrisez les ouvertures, les tactiques et les finales. Vous savez expliquer les principes fondamentaux du jeu et analyser des parties. Vous vous intéressez également à l'impact de l'intelligence artificielle sur les échecs et partagez des conseils pour progresser et affiner sa pensée stratégique.""",
]


# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
client = chromadb.PersistentClient(path="./chromadb")

# Create collection. get_collection, get_or_create_collection, delete_collection also available!
collection = client.create_collection("all-my-documents")

# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=docs, # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    metadatas=[{"source": "notion"}, {"source": "google-docs"}, {"source": "google-docs"}, {"source": "google-docs"}, {"source": "google-docs"}], # filter on these!
    ids=["doc1", "doc2", "doc3", "doc4", "doc5"], # unique for each doc
)

# Query/search 2 most similar results. You can also .get by id
results = collection.query(
    query_texts=["Je veux en savoir plus sur la cuisine et la gastronomie."],
    n_results=2,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)

results

{'ids': [['doc1', 'doc3']],
 'embeddings': None,
 'documents': [["Vous êtes un expert en gastronomie, passionné par la richesse culinaire du monde entier. Vous maîtrisez l'histoire des plats, les techniques de cuisine, les accords mets et vins, ainsi que les spécificités des terroirs. Vous êtes capable d'expliquer l'importance de la cuisine dans la culture d'un pays et de proposer des conseils pour découvrir de nouvelles saveurs.",
   "Vous êtes un passionné de culture, conscient de son rôle fondamental dans la société. Vous explorez les différentes formes d'expression artistique, de la peinture à la littérature, en passant par le cinéma et la musique. Vous valorisez le métissage culturel et comprenez l'importance de préserver et célébrer le patrimoine mondial."]],
 'uris': None,
 'data': None,
 'metadatas': [[{'source': 'notion'}, {'source': 'google-docs'}]],
 'distances': [[0.49239534955773984, 1.1958780244524794]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.d